# Para rodar no databricks

In [ ]:
## limpando caso tenha sido rodado antes e baixado do github
!rm -r ./ML-anti-fraud

rm: cannot remove './ML-anti-fraud': No such file or directory


In [ ]:
# clonando o repositório para rodar no databrics
!git clone https://github.com/caanpaip/ML-anti-fraud.git
# Para trocar para a branch de desenvolvimento onde estão os códigos
!git -C ./ML-anti-fraud checkout develop 

Cloning into 'ML-anti-fraud'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 332 (delta 53), reused 66 (delta 22), pack-reused 214
Receiving objects: 100% (332/332), 80.19 MiB | 7.91 MiB/s, done.
Resolving deltas: 100% (175/175), done.
Updating files: 100% (48/48), done.
Branch 'develop' set up to track remote branch 'develop' from 'origin'.
Switched to a new branch 'develop'


In [ ]:
!ls ML-anti-fraud

README.md  data  docs  models  notebooks  requirements.txt  src


In [ ]:
%pip install optbinning==0.17.0
%pip install scikit-learn==1.1.3
%pip install catboost==1.1.1
%pip install joblib==1.2.0

Python interpreter will be restarted.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.2
    Not uninstalling protobuf at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-fac5bd7e-7241-4030-8711-7086ad9ee043
    Can't uninstall 'protobuf'. No files were found to uninstall.
Python interpreter will be restarted.
Python interpreter will be restarted.
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Not uninstalling scikit-learn at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-fac5bd7e-7241-4030-8711-7086ad9ee043
    Can't uninstall 'scikit-learn'. No files were found to uninstall.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
  Attempting uninstall: joblib
    Found existing installa

#LIB

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import sys
import os
import joblib
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./ML-anti-fraud/src/")


# SPARK

In [ ]:
from pyspark.sql import SparkSession, HiveContext


import pyspark.sql.functions as F
from pyspark.sql.functions import PandasUDFType, pandas_udf
from pyspark.sql.types import DoubleType

spark = SparkSession.builder.appName("sklearn-spark").enableHiveSupport().getOrCreate()

sc = spark.sparkContext
hc = HiveContext(sc)

sc.setLogLevel("ERROR")

In [ ]:
# !ls ML-anti-fraud/src/features

In [ ]:
## validation Dataset
df_validation = pd.read_parquet(f"/databricks/driver/ML-anti-fraud/data/validation_dataset.parquet")
df_validation.drop(['fraude'],axis=1, inplace=True)

In [ ]:
df_validation_spark = spark.createDataFrame(df_validation)

# Modelos

## Regressão logística

>> **Funções do pipeline para scorara regressão logística**

In [ ]:
def list_subset_word(List,word):
    """
    função para imprimir elementos (string) de uma lista que contenham a palavra "word"
    """
    list_subset=[]  
    for ls in List:      
        if word in ls:            
            list_subset.append(ls)       
    return list_subset


def list_subset_words(Lista,list_words):

    """
    função para imprimir elementos de uma lista que contehnam uma lista de palavras
    """

    list_subsets = []

    for word in list_words:

        sub_list = list_subset_word(Lista,word)
        list_subsets.extend(sub_list)

    return [i for n, i in enumerate(list_subsets) if i not in list_subsets[:n]]


def exclude_words(ls:list, words:list)->list:

    """
    Return list excluding elements with containd the word in the list words

        ls:= List to exclude elements
        words: List with word to search and exclude elements.


    """

    ls_exclude = ls.copy()

    for word in words:

        ls_exclude = [x for x in ls_exclude if word not in x]

    return ls_exclude
  
  
def encode_woe_var_transform(df, variable, opt_model_fitted ):

    """
    Return dataframe with new columns using the model fitted.

        df:= Dataframe with columns to transform
        variable:= Variable in dataframe to compute de metrics.
        opt_model_fitted:= Model obtained of 'binning_var_model' function and load using joblib.load(path_model).
    
    
    """
    df_ = df.copy()

    ## model fitted
    opt = opt_model_fitted
    ## Creating WoE table of model fitted
    df_binning = opt.binning_table.build(add_totals=False)
    ## Extracing WoE of missing category using the last row in dataframe
    woe_missing = -df_binning.loc[df_binning.shape[0]-1:,:].reset_index(drop=True)["WoE"][0]
    # Creating encoding using WoE
    df_[f"{variable}_bins"] = opt.transform(df_[variable], metric="indices", metric_missing= -1)
    df_[f"{variable}_bins"] = df_[f"{variable}_bins"].astype("category")
    df_[f"{variable}_cat"] = opt.transform(df_[variable], metric="bins")
    df_[f"{variable}_WoE"] = opt.transform(df_[variable], metric="woe", metric_missing = woe_missing )
    ## Transforming the WoE, If WoE>0 the percentage of event is greather than the Non-event
    df_[f"{variable}_WoE"] = -df_[f"{variable}_WoE"]

    return df_
  
  
def create_woe_columns(df, path_pickle_transf ):

    """ 
    Retorna um dataframe do pandas com colunas adicionáis dada pela tranformação WOE e BINS do para o modelo.

        df:= Dataframe com os dados 
        path_pickle_transf:= Caminho do pickle que tem as tranformações WOE e BIN por cada feature.
    
    
    """

    ## criando uma copia
    dff = df.copy()
    ## Selecionando as colunas para serem usadas
    colunas_orig_and_week = exclude_words( dff.columns.tolist(), ["cat","bin","WoE","produto","fecha","data","hora","week","fraude"] ) + ['categoria_produto']
    ## Iterando para adicionar as colunas WoE
    for variable in colunas_orig_and_week:

        pkl_transform_feature = os.path.join( path_pickle_transf,   f"pipeline_binning__{variable}__.pkl") 

        optb = joblib.load( pkl_transform_feature  )

        dff = encode_woe_var_transform(dff, variable, optb )


    return dff

In [ ]:
cols_rg_model = ['n_boolean_bins_1',
 'p_boolean_bins_1',
 'categoria_produto_bins_1',
 'categoria_produto_bins_2',
 'categoria_produto_bins_3',
 'categoria_produto_bins_4',
 'categoria_produto_bins_5',
 'categoria_produto_bins_6',
 'categoria_produto_bins_7',
 'categoria_produto_bins_8',
 'categoria_produto_bins_9',
 'categoria_produto_bins_10',
 'pais_bins_0',
 'pais_bins_1',
 'a_int_bins_1',
 'b_float_bins_0',
 'b_float_bins_1',
 'b_float_bins_2',
 'b_float_bins_3',
 'b_float_bins_4',
 'b_float_bins_5',
 'c_float_bins_0',
 'c_float_bins_1',
 'd_float_bins_0',
 'd_float_bins_1',
 'd_float_bins_2',
 'd_float_bins_3',
 'd_float_bins_4',
 'd_float_bins_5',
 'd_float_bins_6',
 'e_float_bins_1',
 'e_float_bins_2',
 'f_float_bins_0',
 'f_float_bins_1',
 'f_float_bins_2',
 'f_float_bins_3',
 'f_float_bins_4',
 'f_float_bins_5',
 'f_float_bins_6',
 'f_float_bins_7',
 'f_float_bins_8',
 'f_float_bins_9',
 'h_float_bins_1',
 'h_float_bins_2',
 'h_float_bins_3',
 'h_float_bins_4',
 'h_float_bins_5',
 'k_float_bins_1',
 'k_float_bins_2',
 'k_float_bins_3',
 'k_float_bins_4',
 'l_float_bins_0',
 'l_float_bins_1',
 'l_float_bins_2',
 'l_float_bins_3',
 'l_float_bins_4',
 'l_float_bins_5',
 'l_float_bins_6',
 'l_float_bins_7',
 'l_float_bins_8',
 'l_float_bins_9',
 'l_float_bins_10',
 'l_float_bins_11',
 'm_float_bins_0',
 'm_float_bins_1',
 'm_float_bins_2',
 'm_float_bins_3',
 'm_float_bins_4',
 'm_float_bins_5',
 'm_float_bins_6',
 'm_float_bins_7',
 'm_float_bins_8',
 'm_float_bins_9',
 'm_float_bins_10',
 'm_float_bins_11',
 'monto_bins_1',
 'monto_bins_2',
 'monto_bins_3',
 'monto_bins_4',
 'monto_bins_5',
 'fraude']

In [ ]:
def df_features_bins(df, path_pickle_transf ):

    """ 
    Retorna um dataframe do pandas com colunas que tem os bins de cada feature.

        df:= Dataframe com os dados.
        path_pickle_transf:= Caminho do pickle que tem as tranformações WOE e BIN por cada feature.
    
    
    """    

    df_woe_bins_features = create_woe_columns(df, path_pickle_transf )

    features_bins_woe = list_subset_words( df_woe_bins_features.columns.tolist(), ["bin"])
    ## Excluding variable 'score' (score of Meli model) and 'o_obj' (with 72.5% of missing values))
    features_bins_woe = exclude_words(features_bins_woe, ["score","o_obj"])
    
    try:
        df_bins = df_woe_bins_features[features_bins_woe + ["fraude"]]
    except Exception as e:

        print(e)

        df_bins = df_woe_bins_features[features_bins_woe]

    return df_bins


def transform_2_dummies( df , label = "fraude", drop_first = True):

    """
    Retorno o dataframe com as variáveis dummies para o modelo.

        df:= Dataframe com as variáveis com prefixo bins
        label:= Variável resposta
        drop_first:= Elimina as dummies redundantes

    
    """

    features =list_subset_words( df.columns.tolist(), ["bin"])
    features = exclude_words(features, ["score","o_obj"])

    ls_df_dummies = []

    for var in features:

        ## creating dummy to variable
        df_dummy = pd.get_dummies( df[var] , prefix=var, drop_first = drop_first)
        ls_df_dummies.append(df_dummy)

    try:
        df_2_model_dum = pd.concat(ls_df_dummies + [ df[label] ], axis=1)
    except Exception as e:

        print(e)

        df_2_model_dum = pd.concat(ls_df_dummies , axis=1)

    return df_2_model_dum



def transform_2_model(df ,  label ="fraude", cols_model = cols_rg_model ):

    """
    Retorna o dataframe de entrada do modelo de regressão logísticas.transform_2_dummies

        df:= Dataframe com os campos com prefixo bin
        label:= Variável resposta
        col_modes:= Campos usados no treinamento do modelo

    """

    ## tirando a coluna label caso esteja na lista
    cols_model = exclude_words(cols_model, [label] )
    ## Transformando para dummies
    df_marco_model_dum = transform_2_dummies(df = df, label = label)

    ## Preenchendo colunas faltantes
    dummies_faltantes = list(   set(cols_model) - set(df_marco_model_dum.columns.to_list()) )

    if len(dummies_faltantes)>0:

        for dummi_falt in dummies_faltantes:

            df_marco_model_dum[dummi_falt] = 0

    try:

        df_dummies_features = df_marco_model_dum[cols_model+[label]]

    except Exception as e:

        print(e)

        df_dummies_features = df_marco_model_dum[cols_model]

    return df_dummies_features


def scoring_with_lg(df, path_pickle_transf, path_model   ):
    """
    Retorna o dataframe com a scoragem feita pela regressão logistica treinada

        df:= Dataframe do pandas com os dados de entrada do modelo.
        path_pickle_transf:= Caminho onde encontra-se o pkl para tranformas as variáveis em bins.
        path_model:= Caminho onde encontra-se o pkl do modelo da regressão logística.

    """

    df_ = df.copy()


    ## bins
    df_bins_lg = df_features_bins(df_, path_pickle_transf )
    ## df 2 model
    df_2_model = transform_2_model(df_bins_lg ,  label ="fraude", cols_model = cols_rg_model )
    ## Carregando o modelo
    model_lg = joblib.load( os.path.join( path_model , f"model_lg.pkl") )
    ## Colocando o score
    df_['probability'] = model_lg.predict_proba(df_2_model)[:,1]

    return df_

In [ ]:
path_pickle_transf="/databricks/driver/ML-anti-fraud/src/features"
path_model="/databricks/driver/ML-anti-fraud/models"


df_validation_scored_lg = scoring_with_lg(df_validation, path_pickle_transf, path_model )
df_validation_scored_lg



"['fraude'] not in index"
'fraude'
"['fraude'] not in index"


,a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,data,hora,probability
0,4,0.5893,311762.23,6.0,0.000000,15.0,AR,2,Soldadora Inverter 180 A + Máscara + 2 Esc. Ma...,cat_bfe5d9b,0.897001,4661.0,826.0,1,Y,Y,2020-04-20 22:58:52,142.71,64,2020-04-20,22:58:52,0.313653
1,4,0.7206,227874.02,1.0,0.230960,0.0,BR,8,Par Lanterna Luz De Placa C/ Led Fiat 500 Cone...,cat_768556e,0.530758,2640.0,0.0,1,None,N,2020-04-19 17:07:37,23.79,24,2020-04-19,17:07:37,0.922787
2,4,0.7072,171679.52,1.0,0.592092,101.0,AR,15,Canasto Organizador Ratan Plastico Mediano Tap...,cat_82dfc13,0.656695,5311.0,6.0,1,Y,N,2020-04-20 17:45:23,11.62,91,2020-04-20,17:45:23,0.018533
3,1,0.9315,1576965.58,29.0,0.028381,150.0,BR,2,Motorola Moto E E6 Play Dual Sim 32 Gb Steel B...,cat_43b9c10,0.503253,2678.0,36.0,1,N,N,2020-04-19 11:22:32,117.96,63,2020-04-19,11:22:32,0.726384
4,4,0.8058,330078.07,50.0,0.000000,16.0,BR,17,Jogo Junta Motor Superior Ssangyong Kyron 2.0 ...,cat_d69bf36,0.214595,2525.0,96.0,1,None,Y,2020-04-20 12:38:03,122.49,40,2020-04-20,12:38:03,0.027789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20582,4,0.7099,394368.86,40.0,0.257503,29.0,BR,40,Par Farol Auxiliar De Milha Corsa 2000 2001 2002,cat_0f6c6e1,0.298363,1112.0,84.0,1,None,N,2020-04-18 21:07:53,10.63,29,2020-04-18,21:07:53,0.389763
20583,4,0.7823,68712.34,50.0,0.472789,36.0,BR,16,Banqueta Plastica Dobravel Manicure Pesca Camp...,cat_960e53d,0.196138,3665.0,421.0,1,None,Y,2020-04-21 06:36:50,5.67,75,2020-04-21,06:36:50,0.115280
20584,4,0.7610,15019.78,6.0,0.226857,13.0,BR,1,Chaleira Aço Inox Alça Térmica Com Apito 2 Litros,cat_957d3b1,0.841144,1951.0,49.0,1,None,N,2020-04-21 09:27:00,12.67,63,2020-04-21,09:27:00,0.030181
20585,4,0.7841,82911.05,50.0,0.250185,8.0,AR,9,Tester Digital Mt-1210 Proskit C/ Luz De Fondo...,cat_f6ae276,0.800809,768.0,764.0,1,Y,Y,2020-04-20 21:14:09,19.94,31,2020-04-20,21:14:09,0.022886


## Spark with sklearn - logistic regression

In [ ]:
display(df_validation_spark)

In [ ]:
@pandas_udf("double")
def scoring_with_lg_spark(a_int, b_float, c_float, d_float, e_float, f_float, pais, h_float, produto, categoria_produto, k_float, l_float, m_float, n_boolean, o_obj, p_boolean, fecha, monto, score, data, hora):
  
  import pandas as pd
  os.environ["ARROW_PRE_0_15_IPC_FORMAT"] = "1"
  
  df = pd.concat([a_int, b_float, c_float, d_float, e_float, f_float, pais, h_float, produto, categoria_produto, k_float, l_float, m_float, n_boolean, o_obj, p_boolean, fecha, monto, score, data, hora], axis=1)
  
  df.columns = colunas
  
  df['probability'] = scoring_with_lg(df, path_pickle_transf, path_model )['probability']
  
  return df['probability']  

In [ ]:
colunas = ['a_int', 'b_float', 'c_float', 'd_float', 'e_float', 'f_float', 'pais', 'h_float', 'produto', 'categoria_produto', 'k_float', 'l_float', 'm_float', 'n_boolean', 'o_obj', 'p_boolean', 'fecha', 'monto', 'score', 'data', 'hora']

In [ ]:
df_validation_spark

Out[34]: DataFrame[a_int: bigint, b_float: double, c_float: double, d_float: double, e_float: double, f_float: double, pais: string, h_float: bigint, produto: string, categoria_produto: string, k_float: double, l_float: double, m_float: double, n_boolean: bigint, o_obj: string, p_boolean: string, fecha: string, monto: double, score: bigint, data: timestamp, hora: string]

In [ ]:
display( df_validation_spark.withColumn("probability", scoring_with_lg_spark(*colunas )) )

a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,data,hora,probability
4,0.5893,311762.23,6.0,0.0,15.0,AR,2,Soldadora Inverter 180 A + Máscara + 2 Esc. Mag 4´ Kroner,cat_bfe5d9b,0.8970011894573879,4661.0,826.0,1,Y,Y,2020-04-20 22:58:52,142.71,64,2020-04-20T00:00:00.000+0000,22:58:52,0.31365310217348114
4,0.7206,227874.02,1.0,0.2309602649006622,0.0,BR,8,Par Lanterna Luz De Placa C/ Led Fiat 500 Conector Quadrado,cat_768556e,0.530758419506661,2640.0,0.0,1,null,N,2020-04-19 17:07:37,23.79,24,2020-04-19T00:00:00.000+0000,17:07:37,0.92278687473483
4,0.7072,171679.52,1.0,0.5920921052631578,101.0,AR,15,Canasto Organizador Ratan Plastico Mediano Tapa - Colombraro,cat_82dfc13,0.656695213318418,5311.0,6.0,1,Y,N,2020-04-20 17:45:23,11.62,91,2020-04-20T00:00:00.000+0000,17:45:23,0.01853306887395276
1,0.9315,1576965.58,29.0,0.028380634390651,150.0,BR,2,Motorola Moto E E6 Play Dual Sim 32 Gb Steel Black 2 Gb Ram,cat_43b9c10,0.5032534048088628,2678.0,36.0,1,N,N,2020-04-19 11:22:32,117.96,63,2020-04-19T00:00:00.000+0000,11:22:32,0.7263837030193163
4,0.8058,330078.07,50.0,0.0,16.0,BR,17,Jogo Junta Motor Superior Ssangyong Kyron 2.0 16v 2011,cat_d69bf36,0.2145946983490567,2525.0,96.0,1,null,Y,2020-04-20 12:38:03,122.49,40,2020-04-20T00:00:00.000+0000,12:38:03,0.02778891827670246
4,0.7368,34546.77,23.0,0.2045014520813165,1.0,BR,8,Torneira Gourmet Cozinha Mesa 2 Jatos Flexível Preta,cat_7dc9c03,0.2302032353583918,49.0,46.0,0,null,N,2020-04-17 12:56:40,16.27,91,2020-04-17T00:00:00.000+0000,12:56:40,0.8301526038426549
4,0.732,2158.21,50.0,0.5457317073170731,8.0,BR,2,Kit 12 Lapiseiras De Trocar Ponta Speed C/11 Pontas Cada,cat_8dfbe72,0.504186009520847,2564.0,856.0,1,null,Y,2020-04-18 12:37:06,6.46,6,2020-04-18T00:00:00.000+0000,12:37:06,0.01828178790145439
4,0.6023,179792.69,50.0,0.1951851851851851,62.0,AR,19,Tiraband Circular Pack De 3 Tensiones Importada Oferta,cat_b7ad66e,0.5860484106029274,5631.0,1507.0,1,null,Y,2020-04-19 00:01:06,15.33,6,2020-04-19T00:00:00.000+0000,00:01:06,0.010851030416405666
4,0.6126,129704.3,6.0,0.0,9.0,US,0,Polacrin Talento Interior Exterior Lavable X20 Kgs,cat_851cfbb,0.0907340860579317,320.0,92.0,1,null,N,2020-04-17 15:18:11,53.61,68,2020-04-17T00:00:00.000+0000,15:18:11,0.05560059790371768
4,0.7785,10421.51,44.0,0.9232804232804231,10.0,BR,3,Bolas Suiça 65 Cm Pilates Ginástica Yoga Academia,cat_6b75ad3,0.6136221682594857,3669.0,509.0,1,null,Y,2020-04-20 22:13:38,7.44,37,2020-04-20T00:00:00.000+0000,22:13:38,0.1414849737124309


## Catboost

>> **Funções para escorar o catboost**

In [ ]:
def df_features_cat(df, path_pickle_transf ):

    """ 
    Retorna um dataframe do pandas com colunas que tem as categorias de cada feature.

        df:= Dataframe com os dados.
        path_pickle_transf:= Caminho do pickle que tem as tranformações WOE e BIN por cada feature.
    
    
    """    

    df_woe_bins_features = create_woe_columns(df, path_pickle_transf )

    ## Selecionando as colunas de categorias criadas pelo tranformador anterior
    cat_features = [x for x in list_subset_words( df_woe_bins_features.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
    ## Excluindo as colunas de score, o_obj e data caso existam
    cat_features = exclude_words(cat_features, ["score","o_obj","data"])
    
    try:
        df_2_model_cat = df_woe_bins_features[ cat_features + ["fraude"] ]
    except Exception as e:
        print(e)
        df_2_model_cat = df_woe_bins_features[ cat_features ]


    return df_2_model_cat

def scoring_with_cat( df , path_pickle_transf  , path_model  ):

    df_ = df.copy()
    ## tranformando e selecionando variáveis do modelo do catboost
    df_cat = df_features_cat( df_, path_pickle_transf )
    ## chamando o modelo do catboost salvo no pkl
    modelo_cat = joblib.load( os.path.join( path_model , f"model_cat.pkl") )
    # scorando usando o cat model
    df_['probability'] = modelo_cat.predict_proba(df_cat)[:,1]

    return df_

In [ ]:
df_validation_scored_cat = scoring_with_cat(df_validation, path_pickle_transf, path_model)
df_validation_scored_cat

"['fraude'] not in index"


,a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,data,hora,probability
0,4,0.5893,311762.23,6.0,0.000000,15.0,AR,2,Soldadora Inverter 180 A + Máscara + 2 Esc. Ma...,cat_bfe5d9b,0.897001,4661.0,826.0,1,Y,Y,2020-04-20 22:58:52,142.71,64,2020-04-20,22:58:52,0.246785
1,4,0.7206,227874.02,1.0,0.230960,0.0,BR,8,Par Lanterna Luz De Placa C/ Led Fiat 500 Cone...,cat_768556e,0.530758,2640.0,0.0,1,None,N,2020-04-19 17:07:37,23.79,24,2020-04-19,17:07:37,0.816498
2,4,0.7072,171679.52,1.0,0.592092,101.0,AR,15,Canasto Organizador Ratan Plastico Mediano Tap...,cat_82dfc13,0.656695,5311.0,6.0,1,Y,N,2020-04-20 17:45:23,11.62,91,2020-04-20,17:45:23,0.025566
3,1,0.9315,1576965.58,29.0,0.028381,150.0,BR,2,Motorola Moto E E6 Play Dual Sim 32 Gb Steel B...,cat_43b9c10,0.503253,2678.0,36.0,1,N,N,2020-04-19 11:22:32,117.96,63,2020-04-19,11:22:32,0.698228
4,4,0.8058,330078.07,50.0,0.000000,16.0,BR,17,Jogo Junta Motor Superior Ssangyong Kyron 2.0 ...,cat_d69bf36,0.214595,2525.0,96.0,1,None,Y,2020-04-20 12:38:03,122.49,40,2020-04-20,12:38:03,0.026558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20582,4,0.7099,394368.86,40.0,0.257503,29.0,BR,40,Par Farol Auxiliar De Milha Corsa 2000 2001 2002,cat_0f6c6e1,0.298363,1112.0,84.0,1,None,N,2020-04-18 21:07:53,10.63,29,2020-04-18,21:07:53,0.388800
20583,4,0.7823,68712.34,50.0,0.472789,36.0,BR,16,Banqueta Plastica Dobravel Manicure Pesca Camp...,cat_960e53d,0.196138,3665.0,421.0,1,None,Y,2020-04-21 06:36:50,5.67,75,2020-04-21,06:36:50,0.128385
20584,4,0.7610,15019.78,6.0,0.226857,13.0,BR,1,Chaleira Aço Inox Alça Térmica Com Apito 2 Litros,cat_957d3b1,0.841144,1951.0,49.0,1,None,N,2020-04-21 09:27:00,12.67,63,2020-04-21,09:27:00,0.032461
20585,4,0.7841,82911.05,50.0,0.250185,8.0,AR,9,Tester Digital Mt-1210 Proskit C/ Luz De Fondo...,cat_f6ae276,0.800809,768.0,764.0,1,Y,Y,2020-04-20 21:14:09,19.94,31,2020-04-20,21:14:09,0.033069


## Spark with catboost

In [ ]:
display(df_validation_spark)

In [ ]:
@pandas_udf("double")
def scoring_with_cat_spark(a_int, b_float, c_float, d_float, e_float, f_float, pais, h_float, produto, categoria_produto, k_float, l_float, m_float, n_boolean, o_obj, p_boolean, fecha, monto, score, data, hora):
  
  import pandas as pd
  os.environ["ARROW_PRE_0_15_IPC_FORMAT"] = "1"
  
  df = pd.concat([a_int, b_float, c_float, d_float, e_float, f_float, pais, h_float, produto, categoria_produto, k_float, l_float, m_float, n_boolean, o_obj, p_boolean, fecha, monto, score, data, hora], axis=1)
  
  df.columns = colunas
  
  df['probability'] = scoring_with_cat(df, path_pickle_transf, path_model )['probability']
  
  return df['probability']

In [ ]:
colunas = ['a_int', 'b_float', 'c_float', 'd_float', 'e_float', 'f_float', 'pais', 'h_float', 'produto', 'categoria_produto', 'k_float', 'l_float', 'm_float', 'n_boolean', 'o_obj', 'p_boolean', 'fecha', 'monto', 'score', 'data', 'hora']

In [ ]:
df_validation_spark

Out[38]: DataFrame[a_int: bigint, b_float: double, c_float: double, d_float: double, e_float: double, f_float: double, pais: string, h_float: bigint, produto: string, categoria_produto: string, k_float: double, l_float: double, m_float: double, n_boolean: bigint, o_obj: string, p_boolean: string, fecha: string, monto: double, score: bigint, data: timestamp, hora: string]

In [ ]:
display( df_validation_spark.withColumn("probability", scoring_with_cat_spark(*colunas )) )

a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,data,hora,probability
4,0.5893,311762.23,6.0,0.0,15.0,AR,2,Soldadora Inverter 180 A + Máscara + 2 Esc. Mag 4´ Kroner,cat_bfe5d9b,0.8970011894573879,4661.0,826.0,1,Y,Y,2020-04-20 22:58:52,142.71,64,2020-04-20T00:00:00.000+0000,22:58:52,0.24678541542995006
4,0.7206,227874.02,1.0,0.2309602649006622,0.0,BR,8,Par Lanterna Luz De Placa C/ Led Fiat 500 Conector Quadrado,cat_768556e,0.530758419506661,2640.0,0.0,1,null,N,2020-04-19 17:07:37,23.79,24,2020-04-19T00:00:00.000+0000,17:07:37,0.8164983417563912
4,0.7072,171679.52,1.0,0.5920921052631578,101.0,AR,15,Canasto Organizador Ratan Plastico Mediano Tapa - Colombraro,cat_82dfc13,0.656695213318418,5311.0,6.0,1,Y,N,2020-04-20 17:45:23,11.62,91,2020-04-20T00:00:00.000+0000,17:45:23,0.025566480904994474
1,0.9315,1576965.58,29.0,0.028380634390651,150.0,BR,2,Motorola Moto E E6 Play Dual Sim 32 Gb Steel Black 2 Gb Ram,cat_43b9c10,0.5032534048088628,2678.0,36.0,1,N,N,2020-04-19 11:22:32,117.96,63,2020-04-19T00:00:00.000+0000,11:22:32,0.6982279368482622
4,0.8058,330078.07,50.0,0.0,16.0,BR,17,Jogo Junta Motor Superior Ssangyong Kyron 2.0 16v 2011,cat_d69bf36,0.2145946983490567,2525.0,96.0,1,null,Y,2020-04-20 12:38:03,122.49,40,2020-04-20T00:00:00.000+0000,12:38:03,0.026558236717436557
4,0.7368,34546.77,23.0,0.2045014520813165,1.0,BR,8,Torneira Gourmet Cozinha Mesa 2 Jatos Flexível Preta,cat_7dc9c03,0.2302032353583918,49.0,46.0,0,null,N,2020-04-17 12:56:40,16.27,91,2020-04-17T00:00:00.000+0000,12:56:40,0.8325348653676156
4,0.732,2158.21,50.0,0.5457317073170731,8.0,BR,2,Kit 12 Lapiseiras De Trocar Ponta Speed C/11 Pontas Cada,cat_8dfbe72,0.504186009520847,2564.0,856.0,1,null,Y,2020-04-18 12:37:06,6.46,6,2020-04-18T00:00:00.000+0000,12:37:06,0.028453607834690538
4,0.6023,179792.69,50.0,0.1951851851851851,62.0,AR,19,Tiraband Circular Pack De 3 Tensiones Importada Oferta,cat_b7ad66e,0.5860484106029274,5631.0,1507.0,1,null,Y,2020-04-19 00:01:06,15.33,6,2020-04-19T00:00:00.000+0000,00:01:06,0.01655046693298514
4,0.6126,129704.3,6.0,0.0,9.0,US,0,Polacrin Talento Interior Exterior Lavable X20 Kgs,cat_851cfbb,0.0907340860579317,320.0,92.0,1,null,N,2020-04-17 15:18:11,53.61,68,2020-04-17T00:00:00.000+0000,15:18:11,0.05882506981638313
4,0.7785,10421.51,44.0,0.9232804232804231,10.0,BR,3,Bolas Suiça 65 Cm Pilates Ginástica Yoga Academia,cat_6b75ad3,0.6136221682594857,3669.0,509.0,1,null,Y,2020-04-20 22:13:38,7.44,37,2020-04-20T00:00:00.000+0000,22:13:38,0.18075764398846056
